Synthesize data from all analyses 

WHAT THIS DOES:
- Summarizes teen behavioral risk exposure (Notebook 1)
- Summarizes legislative classifications (Notebooks 3 & 4)
- Presents governance categories, issue tags, mechanisms, and paradigms
- Shows state-by-state breakdown

In [5]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

In [6]:
print("\n\n" + "="*80)
print("1. TEEN BEHAVIORAL RISK EXPOSURE (2017-2024)")
print("="*80)

df_teen = pl.read_csv("../data/teen_e_safety_dataset.csv")
df_teen_us = df_teen.filter(pl.col('Geolocation') == 'US')
df_teen_us = df_teen_us.with_columns([
    pl.col('Timestamp').str.slice(0, 4).cast(pl.Int32).alias('Year')
])

total_sessions = len(df_teen_us)
print(f"\nTotal US teen sessions analyzed: {total_sessions:,}")
print(f"Time period: 2017-2024")

risk_metrics = {
    'Malware_Detection': 'Malware Exposure',
    'Phishing_Attempts': 'Phishing Attempts',
    'Risky_Website_Visits': 'Risky Website Visits', 
    'Cyberbullying_Reports': 'Cyberbullying Reports'
}

print("\nRisk Exposure Rates:")
for col, label in risk_metrics.items():
    if 'Attempts' in col or 'Reports' in col:
        count = len(df_teen_us.filter(pl.col(col) > 0))
    else:
        count = df_teen_us[col].sum()
    
    rate = count / total_sessions * 100
    print(f"  {label:30} {count:6,} sessions ({rate:5.1f}%)")



1. TEEN BEHAVIORAL RISK EXPOSURE (2017-2024)

Total US teen sessions analyzed: 40,807
Time period: 2017-2024

Risk Exposure Rates:
  Malware Exposure                1,982 sessions (  4.9%)
  Phishing Attempts              10,452 sessions ( 25.6%)
  Risky Website Visits            4,048 sessions (  9.9%)
  Cyberbullying Reports             772 sessions (  1.9%)


In [3]:
total_sessions = len(df_teen_us)

risk_metrics = {
    'Malware_Detection': 'Malware Exposure',
    'Phishing_Attempts': 'Phishing Attempts',
    'Risky_Website_Visits': 'Risky Website Visits', 
    'Cyberbullying_Reports': 'Cyberbullying Reports'
}

teen_risks = {}
print("\nTeen Risk Exposure (US Sessions 2017-2024):")
for col, label in risk_metrics.items():
    if 'Attempts' in col or 'Reports' in col:
        count = len(df_teen_us.filter(pl.col(col) > 0))
    else:
        count = df_teen_us[col].sum()
    
    rate = count / total_sessions * 100
    teen_risks[label] = rate
    print(f"  {label:25} {rate:5.1f}% of sessions")


Teen Risk Exposure (US Sessions 2017-2024):
  Malware Exposure            4.9% of sessions
  Phishing Attempts          25.6% of sessions
  Risky Website Visits        9.9% of sessions
  Cyberbullying Reports       1.9% of sessions


In [9]:
print("\n\n" + "="*80)
print("2. STATE CHILD SAFETY LEGISLATION DATASET")
print("="*80)

df_policy = pl.read_csv('../data/bill_classifications_full.csv')

print(f"\nTotal bills analyzed: {len(df_policy)}")
print(f"States represented: {df_policy['state'].n_unique()}")
print(f"Time period: 2021-2023 (majority)")



2. STATE CHILD SAFETY LEGISLATION DATASET

Total bills analyzed: 146
States represented: 43
Time period: 2021-2023 (majority)


In [10]:
print("\n\n" + "="*80)
print("3. GOVERNANCE CATEGORIES")
print("="*80)

governance_counts = (
    df_policy
    .group_by('governance')
    .agg(pl.len().alias('count'))
    .sort('count', descending=True)
)

print("\nBills by governance category:")
for row in governance_counts.iter_rows(named=True):
    gov = row['governance'] if row['governance'] else 'Unlabeled'
    count = row['count']
    pct = count / len(df_policy) * 100
    print(f"  {gov:40} {count:3} bills ({pct:5.1f}%)")



3. GOVERNANCE CATEGORIES

Bills by governance category:
  Content Moderation/Free Speech            52 bills ( 35.6%)
  Privacy/Data Rights                       39 bills ( 26.7%)
  Platforms + Democracy                     25 bills ( 17.1%)
  Government Surveillance                   19 bills ( 13.0%)
  Content moderation/Free Speech            11 bills (  7.5%)


In [11]:
print("\n\n" + "="*80)
print("4. ISSUE TAGS")
print("="*80)

# Collect all issue tags
all_tags = []
for col in ['issue_tag_1', 'issue_tag_2', 'issue_tag_3']:
    tags = df_policy.filter(pl.col(col).is_not_null()).select(col).to_series().to_list()
    all_tags.extend(tags)

tag_counter = Counter(all_tags)

print(f"\nMost common issue tags (Top 15):")
for tag, count in tag_counter.most_common(15):
    pct = count / len(df_policy) * 100
    print(f"  {tag:40} {count:3} bills ({pct:5.1f}%)")



4. ISSUE TAGS

Most common issue tags (Top 15):
  Online safety                            109 bills ( 74.7%)
  Privacy                                   77 bills ( 52.7%)
  Content moderation/Free Speech            51 bills ( 34.9%)
  Liability                                 28 bills ( 19.2%)
  Design and testing standards              23 bills ( 15.8%)
  Education and research                    20 bills ( 13.7%)
  Transparency                              15 bills ( 10.3%)
  Algorithms                                14 bills (  9.6%)
  Cybersecurity                             12 bills (  8.2%)
  Authorisation, registration and licensing  12 bills (  8.2%)
  Internet access                            8 bills (  5.5%)
  Hate and harassment                        6 bills (  4.1%)
  Taxation                                   4 bills (  2.7%)
  Human rights                               3 bills (  2.1%)
  Civil rights                               3 bills (  2.1%)


In [12]:
print("\n\n" + "="*80)
print("5. POLICY MECHANISMS (9 Dimensions)")
print("="*80)

mechanism_names = {
    'age_verification': 'Age Verification',
    'parental_consent': 'Parental Consent',
    'data_collection_limits': 'Data Collection Limits',
    'algorithmic_restrictions': 'Algorithmic Restrictions',
    'duty_of_care': 'Duty of Care',
    'risk_assessment_required': 'Risk Assessment Required',
    'default_privacy_settings': 'Default Privacy Settings',
    'school_based': 'School-Based (Education)',
    'targets_all_platforms': 'Targets All Platforms'
}

print("\nMechanism frequency across 146 bills:")
for mech_key, mech_label in mechanism_names.items():
    count = df_policy[mech_key].sum()
    pct = count / len(df_policy) * 100
    print(f"  {mech_label:35} {count:3} bills ({pct:5.1f}%)")

# Mechanism complexity
df_policy_with_count = df_policy.with_columns([
    pl.sum_horizontal([
        pl.col(mech).cast(pl.Int32) 
        for mech in mechanism_names.keys()
    ]).alias('total_mechanisms')
])

print(f"\nMechanism complexity:")
print(f"  Bills with 0 mechanisms:  {len(df_policy_with_count.filter(pl.col('total_mechanisms') == 0)):3} bills")
print(f"  Bills with 1 mechanism:   {len(df_policy_with_count.filter(pl.col('total_mechanisms') == 1)):3} bills")
print(f"  Bills with 2+ mechanisms: {len(df_policy_with_count.filter(pl.col('total_mechanisms') >= 2)):3} bills")
print(f"  Maximum mechanisms:       {df_policy_with_count['total_mechanisms'].max():3} mechanisms in single bill")



5. POLICY MECHANISMS (9 Dimensions)

Mechanism frequency across 146 bills:
  Age Verification                     40 bills ( 27.4%)
  Parental Consent                     24 bills ( 16.4%)
  Data Collection Limits               70 bills ( 47.9%)
  Algorithmic Restrictions             18 bills ( 12.3%)
  Duty of Care                         13 bills (  8.9%)
  Risk Assessment Required             20 bills ( 13.7%)
  Default Privacy Settings             21 bills ( 14.4%)
  School-Based (Education)             22 bills ( 15.1%)
  Targets All Platforms                39 bills ( 26.7%)

Mechanism complexity:
  Bills with 0 mechanisms:   29 bills
  Bills with 1 mechanism:    31 bills
  Bills with 2+ mechanisms:  86 bills
  Maximum mechanisms:         5 mechanisms in single bill


In [13]:
print("\n\n" + "="*80)
print("6. POLICY PARADIGMS")
print("="*80)

paradigm_counts = (
    df_policy
    .group_by('paradigm')
    .agg(pl.len().alias('count'))
    .sort('count', descending=True)
)

print("\nBills by policy paradigm:")
for row in paradigm_counts.iter_rows(named=True):
    count = row['count']
    pct = count / len(df_policy) * 100
    print(f"  {row['paradigm']:30} {count:3} bills ({pct:5.1f}%)")



6. POLICY PARADIGMS

Bills by policy paradigm:
  Privacy Protection              50 bills ( 34.2%)
  Procedural/Study                32 bills ( 21.9%)
  Platform Responsibility         22 bills ( 15.1%)
  Other/Mixed                     16 bills ( 11.0%)
  Educational                     13 bills (  8.9%)
  Access Control                   8 bills (  5.5%)
  Comprehensive/Hybrid             5 bills (  3.4%)


In [14]:
print("\n\n" + "="*80)
print("7. STATE-BY-STATE ANALYSIS")
print("="*80)

state_summary = (
    df_policy
    .group_by('state')
    .agg([
        pl.len().alias('total_bills'),
        pl.col('age_verification').sum().alias('age_verify'),
        pl.col('data_collection_limits').sum().alias('data_limits'),
        pl.col('algorithmic_restrictions').sum().alias('algo_restrict'),
        pl.col('duty_of_care').sum().alias('duty_care'),
        pl.col('school_based').sum().alias('school')
    ])
    .sort('total_bills', descending=True)
)

print("\nTop 15 states by number of bills:")
print(f"{'State':<20} {'Total':>6} {'Age':>5} {'Data':>5} {'Algo':>5} {'Duty':>5} {'School':>5}")
print(f"{'':20} {'Bills':>6} {'Ver':>5} {'Lim':>5} {'Res':>5} {'Care':>5} {'Based':>5}")
print("-" * 80)

for row in state_summary.head(15).iter_rows(named=True):
    print(f"{row['state']:<20} {row['total_bills']:>6} {row['age_verify']:>5} {row['data_limits']:>5} "
          f"{row['algo_restrict']:>5} {row['duty_care']:>5} {row['school']:>5}")

# Show states with most comprehensive bills (3+ mechanisms)
comprehensive_by_state = (
    df_policy_with_count
    .filter(pl.col('total_mechanisms') >= 3)
    .group_by('state')
    .agg(pl.len().alias('comprehensive_bills'))
    .sort('comprehensive_bills', descending=True)
)

print(f"\n\nStates with most comprehensive bills (3+ mechanisms):")
for row in comprehensive_by_state.head(10).iter_rows(named=True):
    print(f"  {row['state']:20} {row['comprehensive_bills']:3} comprehensive bills")



7. STATE-BY-STATE ANALYSIS

Top 15 states by number of bills:
State                 Total   Age  Data  Algo  Duty School
                      Bills   Ver   Lim   Res  Care Based
--------------------------------------------------------------------------------
New York                 13     1     4     2     2     3
Texas                    13     9    10     2     0     1
New Jersey               12     0     2     2     2     2
Massachusetts             9     0     3     0     0     3
Connecticut               7     1     3     1     2     0
Minnesota                 6     1     2     2     2     0
South Carolina            5     3     1     1     0     0
Florida                   5     2     4     1     0     2
Mississippi               5     3     3     0     0     2
California                4     1     2     2     2     0
West Virginia             4     1     3     0     0     0
Maryland                  4     1     2     1     0     0
Iowa                      4     2     2   

In [15]:
print("\n\n" + "="*80)
print("8. DOMINANT PARADIGM BY STATE")
print("="*80)

# For each state, find the most common paradigm
state_paradigms = []
for state in df_policy['state'].unique().sort().to_list():
    state_bills = df_policy.filter(pl.col('state') == state)
    
    if len(state_bills) > 0:
        paradigm_counts_state = (
            state_bills
            .group_by('paradigm')
            .agg(pl.len().alias('count'))
            .sort('count', descending=True)
        )
        
        top_paradigm = paradigm_counts_state[0, 'paradigm']
        top_count = paradigm_counts_state[0, 'count']
        
        state_paradigms.append({
            'state': state,
            'total_bills': len(state_bills),
            'dominant_paradigm': top_paradigm,
            'paradigm_count': top_count
        })

df_state_paradigms = pl.DataFrame(state_paradigms).sort('total_bills', descending=True)

print("\nDominant paradigm by state (Top 15):")
print(f"{'State':<20} {'Total Bills':>12} {'Dominant Paradigm':<30}")
print("-" * 80)

for row in df_state_paradigms.head(15).iter_rows(named=True):
    print(f"{row['state']:<20} {row['total_bills']:>12} {row['dominant_paradigm']:<30}")




8. DOMINANT PARADIGM BY STATE

Dominant paradigm by state (Top 15):
State                 Total Bills Dominant Paradigm             
--------------------------------------------------------------------------------
New York                       13 Procedural/Study              
Texas                          13 Privacy Protection            
New Jersey                     12 Procedural/Study              
Massachusetts                   9 Procedural/Study              
Connecticut                     7 Platform Responsibility       
Minnesota                       6 Platform Responsibility       
Florida                         5 Privacy Protection            
Mississippi                     5 Privacy Protection            
South Carolina                  5 Access Control                
California                      4 Other/Mixed                   
Iowa                            4 Privacy Protection            
Maryland                        4 Platform Responsibility       
West

In [19]:
# ===== 9. BILL STATUS =====
print("\n\n" + "="*80)
print("9. BILL STATUS")
print("="*80)

# Reload original data to get status information
df_original = pl.read_excel(
    "../data/policy_tracker.xlsx", 
    sheet_name="4. US State - Current Child"
)

# Merge status with our classified bills
df_with_status = df_policy.join(
    df_original.select(['US State', 'Name', 'Status (optional)']),
    left_on=['state', 'name'],
    right_on=['US State', 'Name'],
    how='left'
)

# Count bills by status
status_counts = (
    df_with_status
    .group_by('Status (optional)')
    .agg(pl.len().alias('count'))
    .sort('count', descending=True)
)

print("\nBills by status:")
for row in status_counts.iter_rows(named=True):
    status = row['Status (optional)'] if row['Status (optional)'] else 'Unknown'
    count = row['count']
    pct = count / len(df_with_status) * 100
    print(f"  {str(status)[:50]:50} {count:3} bills ({pct:5.1f}%)")

# Identify passed bills
passed_keywords = ['passed', 'enacted', 'act ', 'chaptered', 'signed', 'effective']
passed_bills = df_with_status.filter(
    pl.col('Status (optional)')
    .cast(pl.Utf8)
    .str.to_lowercase()
    .str.contains('|'.join(passed_keywords))
)

print(f"\n✓ Identified {len(passed_bills)} PASSED bills")

if len(passed_bills) > 0:
    print("\nPassed bills by state:")
    for row in passed_bills.iter_rows(named=True):
        print(f"  • {row['state']:15} {row['name'][:60]}")
        print(f"    Status: {row['Status (optional)']}")
        print(f"    Paradigm: {row['paradigm']}")
        
        # Show mechanisms
        mechs = []
        for mech_key, mech_label in mechanism_names.items():
            if row.get(mech_key):
                mechs.append(mech_label)
        print(f"    Mechanisms: {', '.join(mechs) if mechs else 'None'}")
        print()

Could not determine dtype for column 14, falling back to string




9. BILL STATUS

Bills by status:
  Unknown                                            173 bills ( 92.0%)
  4/11/2023 - Passed, now Act 689 - 100% progression   1 bills (  0.5%)
  5/2/2023 - Tabled for the Calendar, House            1 bills (  0.5%)
  5/17/2023 - Engrossed - 50% progression              1 bills (  0.5%)
  3/9/2022 - Introduced - 25% progression, died in c   1 bills (  0.5%)
  6/1/2023 - Ordered to inactive file on request of    1 bills (  0.5%)
  4/25/2023 - Set for first hearing canceled at the    1 bills (  0.5%)
  1/17/2023 - Referred to Joint Committee on Public    1 bills (  0.5%)
  9/15/2023 - Chaptered by Secretary of State - Chap   1 bills (  0.5%)
  2/28/2023 - Public Hearing                           1 bills (  0.5%)
  3/14/2023 - Public Hearing, Pending Before Joint P   1 bills (  0.5%)
  1/18/2023 - Referred to Joint Committee on General   1 bills (  0.5%)
  3/3/2023 - Engrossed - 50% progression               1 bills (  0.5%)
  4/3/2023 - In committee, Se

In [20]:
# ===== 10. KEY STATISTICS SUMMARY =====
print("\n\n" + "="*80)
print("10. KEY STATISTICS SUMMARY")
print("="*80)

# Get teen risk values from earlier calculation
malware_rate = [rate for label, rate in zip(risk_metrics.values(), 
                [teen_risks[label] for label in risk_metrics.values()]) 
                if 'Malware' in label][0]
phishing_rate = [rate for label, rate in zip(risk_metrics.values(), 
                 [teen_risks[label] for label in risk_metrics.values()]) 
                 if 'Phishing' in label][0]
risky_rate = [rate for label, rate in zip(risk_metrics.values(), 
              [teen_risks[label] for label in risk_metrics.values()]) 
              if 'Risky' in label][0]
cyber_rate = [rate for label, rate in zip(risk_metrics.values(), 
              [teen_risks[label] for label in risk_metrics.values()]) 
              if 'Cyberbullying' in label][0]

print(f"""
TEEN RISKS:
  • Malware exposure: {malware_rate:.1f}% of sessions
  • Phishing attempts: {phishing_rate:.1f}% of sessions
  • Risky website visits: {risky_rate:.1f}% of sessions
  • Cyberbullying reports: {cyber_rate:.1f}% of sessions

LEGISLATIVE LANDSCAPE:
  • Total bills: {len(df_policy)}
  • States: {df_policy['state'].n_unique()}
  • Most active state: {state_summary[0, 'state']} ({state_summary[0, 'total_bills']} bills)
  
DOMINANT APPROACHES:
  • Governance: {governance_counts[0, 'governance']} ({governance_counts[0, 'count']} bills)
  • Paradigm: {paradigm_counts[0, 'paradigm']} ({paradigm_counts[0, 'count']} bills)
  • Mechanism: Data Collection Limits (70 bills, 48%)
  
COMPLEXITY:
  • Bills with no mechanisms: {len(df_policy_with_count.filter(pl.col('total_mechanisms') == 0))} (procedural/study)
  • Bills with 1 mechanism: {len(df_policy_with_count.filter(pl.col('total_mechanisms') == 1))}
  • Bills with 2+ mechanisms: {len(df_policy_with_count.filter(pl.col('total_mechanisms') >= 2))}
""")



10. KEY STATISTICS SUMMARY

TEEN RISKS:
  • Malware exposure: 4.9% of sessions
  • Phishing attempts: 25.6% of sessions
  • Risky website visits: 9.9% of sessions
  • Cyberbullying reports: 1.9% of sessions

LEGISLATIVE LANDSCAPE:
  • Total bills: 146
  • States: 43
  • Most active state: New York (13 bills)

DOMINANT APPROACHES:
  • Governance: Content Moderation/Free Speech (52 bills)
  • Paradigm: Privacy Protection (50 bills)
  • Mechanism: Data Collection Limits (70 bills, 48%)

COMPLEXITY:
  • Bills with no mechanisms: 29 (procedural/study)
  • Bills with 1 mechanism: 31
  • Bills with 2+ mechanisms: 86

